# ZenML Quickstart: From Agent-Only to Agent+Classifier

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/quickstart.ipynb)

**The modern AI development story in 5 minutes.**

This quickstart demonstrates how ZenML unifies ML and Agent workflows, showing the natural progression from a generic agent to a specialized one powered by your own trained models.

## 🎯 The Story

**Phase 1**: Deploy a support agent → Generic responses  
**Phase 2**: Train an intent classifier → Tag as "production"  
**Phase 3**: Same agent automatically upgrades → Specialized responses  

**The "aha" moment**: ZenML connects batch training and real-time serving with the same primitives. Train offline, promote artifacts to production, and agents automatically consume them online.

## 🛠️ Setup

First, let's install ZenML and required dependencies:

In [ ]:
# Install ZenML and dependencies
!pip install "zenml[server]" scikit-learn requests -q

# If running in Colab, we need to restart runtime after installation
import sys

if "google.colab" in sys.modules:
    print(
        "⚠️ Please restart runtime after installation (Runtime → Restart runtime)"
    )
    print("Then continue with the next cell.")

Initialize ZenML and set up the deployer stack:

In [ ]:
import sys

# Initialize ZenML repository
!zenml init

# Set up deployer stack (required for serving)
!zenml deployer register docker -f docker
!zenml stack register docker-deployer -o default -a default -D docker --set

print("✅ ZenML setup complete!")

Download the quickstart files if not already present:

In [ ]:
import urllib.request
from pathlib import Path

# Base URL for quickstart files
base_url = (
    "https://raw.githubusercontent.com/zenml-io/zenml/main/examples/quickstart"
)

# Files to download
files = {
    "run.py": "run.py",
    "configs/agent.yaml": "configs/agent.yaml",
    "pipelines/intent_training_pipeline.py": "pipelines/intent_training_pipeline.py",
    "pipelines/agent_serving_pipeline.py": "pipelines/agent_serving_pipeline.py",
    "steps/data.py": "steps/data.py",
    "steps/train.py": "steps/train.py",
    "steps/infer.py": "steps/infer.py",
    "pipelines/__init__.py": "pipelines/__init__.py",
    "steps/__init__.py": "steps/__init__.py",
}

# Create directories and download files
for file_path, local_path in files.items():
    local_dir = Path(local_path).parent
    local_dir.mkdir(parents=True, exist_ok=True)

    if not Path(local_path).exists():
        try:
            url = f"{base_url}/{file_path}"
            urllib.request.urlretrieve(url, local_path)
            print(f"✅ Downloaded {local_path}")
        except Exception as e:
            print(f"❌ Failed to download {local_path}: {e}")
    else:
        print(f"📁 {local_path} already exists")

print("\n🎉 All files ready!")

## 📋 Phase 1: Deploy Agent-Only

Let's deploy the agent without any classifier - it will use generic responses:

In [ ]:
# Deploy the agent serving pipeline
!zenml pipeline deploy pipelines.agent_serving_pipeline.agent_serving_pipeline \
  -n support-agent -c configs/agent.yaml

print("\n✅ Agent deployed! It's now running with generic responses only.")

Test the agent with a sample banking question:

In [ ]:
import json
import time

import requests

# Wait a moment for deployment to be ready
print("⏳ Waiting for agent to be ready...")
time.sleep(10)

# Test the agent
url = "http://localhost:8000/invoke"
payload = {"parameters": {"text": "my card is lost and i need a replacement"}}

try:
    response = requests.post(
        url, json=payload, headers={"Content-Type": "application/json"}
    )
    result = response.json()

    print("🤖 Agent Response (Phase 1 - Generic):")
    print(json.dumps(result, indent=2))

    # Parse the nested JSON response
    if "output" in result:
        agent_output = json.loads(result["output"])
        print(
            f"\n📊 Intent Source: {agent_output.get('intent_source', 'unknown')}"
        )
        print(f"🎯 Detected Intent: {agent_output.get('intent', 'unknown')}")
        print(f"📈 Confidence: {agent_output.get('confidence', 0):.2f}")

except requests.exceptions.ConnectionError:
    print(
        "❌ Could not connect to agent. Make sure Docker is running and the deployment succeeded."
    )
except Exception as e:
    print(f"❌ Error testing agent: {e}")

## 🧠 Phase 2: Train Intent Classifier

Now let's train an intent classifier and automatically tag it as "production":

In [ ]:
# Run the training pipeline
!python run.py --train

print("\n✅ Training complete! The classifier is now tagged as 'production'.")

Let's verify the artifact was created and tagged:

In [ ]:
from zenml.client import Client

# Check if our production artifact exists
client = Client()
try:
    versions = client.list_artifact_versions(name="intent-classifier")

    print("📦 Intent Classifier Artifacts:")
    for version in versions:
        tags = [tag.name for tag in version.tags] if version.tags else []
        print(f"  Version {version.version}: {tags}")

        if "production" in tags:
            print(f"  ✅ Found production version: {version.version}")
            break
    else:
        print("  ❌ No production-tagged version found")

except Exception as e:
    print(f"❌ Error checking artifacts: {e}")

## 🚀 Phase 3: Agent Automatically Upgrades

Update the same deployment - the agent will now load the production classifier:

In [ ]:
# Update the deployment with the -u flag
!zenml pipeline deploy pipelines.agent_serving_pipeline.agent_serving_pipeline \
  -n support-agent -c configs/agent.yaml -u

print("\n✅ Deployment updated! Agent will now use the production classifier.")

Test with the same request - now it should use the classifier:

In [ ]:
# Wait for updated deployment to be ready
print("⏳ Waiting for updated agent to be ready...")
time.sleep(15)

# Test the upgraded agent
try:
    response = requests.post(
        url, json=payload, headers={"Content-Type": "application/json"}
    )
    result = response.json()

    print("🤖 Agent Response (Phase 3 - Specialized):")
    print(json.dumps(result, indent=2))

    # Parse the nested JSON response
    if "output" in result:
        agent_output = json.loads(result["output"])
        print(
            f"\n📊 Intent Source: {agent_output.get('intent_source', 'unknown')}"
        )
        print(f"🎯 Detected Intent: {agent_output.get('intent', 'unknown')}")
        print(f"📈 Confidence: {agent_output.get('confidence', 0):.2f}")

        # Highlight the upgrade
        if agent_output.get("intent_source") == "classifier":
            print("\n🎉 SUCCESS! Agent is now using the trained classifier!")
        else:
            print(
                "\n⚠️ Agent is still using generic responses. Try again in a moment."
            )

except requests.exceptions.ConnectionError:
    print(
        "❌ Could not connect to agent. Make sure the deployment is running."
    )
except Exception as e:
    print(f"❌ Error testing agent: {e}")

## 🔍 The Magic Explained

Let's look at how the automatic upgrade works:

In [ ]:
# Show the key code that makes this work
print("🪄 The Automatic Upgrade Logic:")
print("""
# At deployment startup (on_init_hook)
def _load_production_classifier_if_any():
    versions = client.list_artifact_versions(name="intent-classifier")

    for version in versions:
        if "production" in [tag.name for tag in version.tags]:
            global _router
            _router = version.load()  # 🎯 Agent upgraded!
            break

# The Decision Logic:
if _router is not None:
    # ✅ Use classifier for specific responses
    intent_source = "classifier"
else:
    # ❌ Use generic LLM responses  
    intent_source = "llm"
""")

print("\n🎯 Key Points:")
print("1. 🚀 Deploy once → Agent works immediately with generic responses")
print("2. 🧠 Train model → Automatically tagged as 'production'")
print("3. 🔄 Update deployment → Agent finds and loads the production model")
print("4. ✨ Same endpoint, better responses → No code changes needed")

## 🧪 Test Different Banking Intents

Let's test the classifier with different banking questions:

In [ ]:
# Test different banking intents
test_cases = [
    "what is my current balance",
    "i need to make a payment",
    "i want to dispute a charge",
    "can you increase my credit limit",
    "hello can you help me",
]

print("🧪 Testing Different Banking Intents:\n")

for i, test_text in enumerate(test_cases, 1):
    print(f"Test {i}: '{test_text}'")

    payload = {"parameters": {"text": test_text}}

    try:
        response = requests.post(
            url, json=payload, headers={"Content-Type": "application/json"}
        )
        result = response.json()

        if "output" in result:
            agent_output = json.loads(result["output"])
            intent = agent_output.get("intent", "unknown")
            confidence = agent_output.get("confidence", 0)
            source = agent_output.get("intent_source", "unknown")

            print(
                f"  → Intent: {intent} (confidence: {confidence:.2f}, source: {source})"
            )
        else:
            print("  → Error: No output in response")

    except Exception as e:
        print(f"  → Error: {e}")

    print()
    time.sleep(1)  # Brief pause between requests

## 🧹 Cleanup

Stop the deployment when you're done:

In [ ]:
# Stop the deployment
try:
    !zenml deployment delete support-agent
    print("✅ Deployment stopped and cleaned up.")
except:
    print("ℹ️ Deployment may have already been stopped.")

## 🎯 What You Just Learned

Congratulations! You've experienced ZenML's unified ML and Agent workflow:

### 🔄 **The Progression**
1. **Generic Agent** → Deployed instantly, works with basic responses
2. **Trained Classifier** → Added specialized intelligence with production tagging
3. **Automatic Upgrade** → Same endpoint, enhanced capabilities

### 🏗️ **ZenML Features Demonstrated**
- ✅ **Unified Workflows**: Same primitives for batch training and real-time serving
- ✅ **Production Tagging**: Automatic artifact promotion with `add_tags()`
- ✅ **Warm Container Serving**: Models load once at startup for fast responses
- ✅ **Artifact Lineage**: Full tracking from training to deployment
- ✅ **Stack Portability**: Deploy anywhere with consistent APIs

### 🌟 **Key Takeaway**
*One framework for ML and Agents. Train offline, promote artifacts into production, and serve online with the same developer experience.*

---

### 🚀 **Next Steps**
- 📖 [Full ZenML Documentation](https://docs.zenml.io/)
- 💬 [Join our Community](https://zenml.io/slack)
- 🏢 [ZenML Pro](https://zenml.io/pro) for teams
- 🌐 [More Examples](https://github.com/zenml-io/zenml/tree/main/examples)

**Ready to build your own AI workflows?** ZenML provides the infrastructure to make them reliable, reproducible, and production-ready.